In [291]:
import pandas as pd
import statistics
import numpy as np

In [292]:
#Import the cvs file as a data frame
rice_scores = pd.read_csv('rice_scores.csv')

print(rice_scores.head())

  product  reach  impact  confidence  effort   RICE  main_tech_dep  \
0      P1    170       1         0.8       4   34.0              1   
1      P2    200       3         0.5       3  100.0              2   
2      P3    220       3         0.5       2  165.0              1   
3      P4     40       3         0.8       5   19.2              4   
4      P5    500       3         0.3       2  225.0              2   

   tech_dep_2  tech_dep_3  
0         2.0         NaN  
1         5.0         3.0  
2         3.0         4.0  
3         5.0         2.0  
4         NaN         NaN  


In [298]:
#Rename columns to have no spaces, all lower case
rice_scores = rice_scores.rename(columns={'Product': 'product', 'Reach': 'reach', 'Impact':'impact',
                                         'Confidence':'confidence', 'Effort':'effort', 'Main Tech Dep':'main_tech_dep',
                                          'Tech Dep 2':'tech_dep_2', 'Tech Dep 3':'tech_dep_3'})

#Add a column for the actual RICE score of each product. Reorder columns so RICE comes before tech dependencies
rice_scores['RICE'] = (rice_scores.reach * rice_scores.impact * rice_scores.confidence)/rice_scores.effort
rice_scores = rice_scores[['product', 'reach', 'impact', 'confidence', 'effort', 'RICE', 'main_tech_dep', 'tech_dep_2', 'tech_dep_3']]

print(rice_scores)

  product  reach  impact  confidence  effort        RICE  main_tech_dep  \
0      P1    170       1         0.8       4   34.000000              1   
1      P2    200       3         0.5       3  100.000000              2   
2      P3    220       3         0.5       2  165.000000              1   
3      P4     40       3         0.8       5   19.200000              4   
4      P5    500       3         0.3       2  225.000000              2   
5      P6     80       3         0.5       2   60.000000              1   
6      P7    300       1         0.8       1  240.000000              5   
7      P8    120       2         0.3       1   72.000000              3   
8      P9     25       3         0.5       2   18.750000              5   
9     P10    200       2         0.5       3   66.666667              5   

   tech_dep_2  tech_dep_3  
0         2.0         NaN  
1         5.0         3.0  
2         3.0         4.0  
3         5.0         2.0  
4         NaN         NaN  
5     

In [299]:
#I don't know if this is an effective way to do this, but I want to know which Products each tech dependency is a dependency for
# so I'm going to create lists to contain this information

product_list = rice_scores['product'].tolist()
tech_dep_1_list = []
tech_dep_2_list = []
tech_dep_3_list = []
tech_dep_4_list = []
tech_dep_5_list = []
tech_deps_lists = [tech_dep_1_list, tech_dep_2_list, tech_dep_3_list, tech_dep_4_list, tech_dep_5_list]
for i in range(5):
    for j in range(len(rice_scores)):
        if rice_scores.loc[j, 'main_tech_dep'] == i+1 or rice_scores.loc[j, 'tech_dep_2'] == i+1 or rice_scores.loc[j, 'tech_dep_3'] == i+1:
            tech_deps_lists[i].append(product_list[j])

#tech_to_products_dict = {'tech_dep_1':tech_dep_1_list, 'tech_dep_2':tech_dep_2_list, 'tech_dep_3':tech_dep_3_list,  
                         #'tech_dep_4':tech_dep_4_list, 'tech_dep_5':tech_dep_5_list,}
# max_length = 0
# for col in tech_to_products_dict.values():
#     if len(col) > max_length:
#         max_length = len(col)

# for col in tech_to_products_dict.values():
#     for _ in range(max_length - len(col)):
#         col.append(np.nan)

max_length = 0
for list in tech_deps_lists:
    if len(list) > max_length:
        max_length = len(list)

for list in tech_deps_lists:
    for _ in range(max_length - len(list)):
        list.append(np.nan)

tech_to_products_dict = {'tech': [1, 2, 3, 4, 5],
                         'dependent_1':[tech_dep_1_list[0], tech_dep_2_list[0], tech_dep_3_list[0], tech_dep_4_list[0], tech_dep_5_list[0]],
                         'dependent_2':[tech_dep_1_list[1], tech_dep_2_list[1], tech_dep_3_list[1], tech_dep_4_list[1], tech_dep_5_list[1]],
                         'dependent_3':[tech_dep_1_list[2], tech_dep_2_list[2], tech_dep_3_list[2], tech_dep_4_list[2], tech_dep_5_list[2]],
                         'dependent_4':[tech_dep_1_list[3], tech_dep_2_list[3], tech_dep_3_list[3], tech_dep_4_list[3], tech_dep_5_list[3]],
                         'dependent_5': [tech_dep_1_list[4], tech_dep_2_list[4], tech_dep_3_list[4], tech_dep_4_list[4], tech_dep_5_list[4]],
                         'dependent_6': [tech_dep_1_list[5], tech_dep_2_list[5], tech_dep_3_list[5], tech_dep_4_list[5], tech_dep_5_list[5]]}

tech_to_product = pd.DataFrame.from_dict(tech_to_products_dict)
print(tech_to_product)

   tech dependent_1 dependent_2 dependent_3 dependent_4 dependent_5  \
0     1          P1          P3          P6          P7         P10   
1     2          P1          P2          P4          P5          P8   
2     3          P2          P3          P7          P8          P9   
3     4          P3          P4          P8         NaN         NaN   
4     5          P2          P4          P6          P7          P9   

  dependent_6  
0         NaN  
1          P9  
2         NaN  
3         NaN  
4         P10  


Next, we are going to define a handful of possible scores that we can use to rank the tech priorities. Here, to simplify things, I've randomly generated numbers 1-5 to stand in for the tech dependencies. The goal is to create a third DataFrame, indexed by the tech dependencies with different possible rankings as the columns.

For ease of writing, assume that tech dependency _n_ appears as a dependency for P1, P2, and P3. We'll create functions for each of the following rank formulas:

$$rank_1(1) = RICE(P1) + RICE(P2) + RICE(P3)$$
$$rank_2(n) = \frac{(R_1 \times I_1 \times C_1) + (R_2 \times I_2 \times C_2) + (R_3 \times I_3 \times C_3)}{E_1 + E_2 + E_3}$$ 
$$rank_3(n) = \frac{(R_1 + R_2 + R_3) \times (I_1 + I_2 + I_3) \times (Avg(C_1, C_2, C_3))}{E_1 + E_2 + E_3}$$
$$rank_4(n) = \frac{(R_1 + R_2 + R_3) \times (I_1 + I_2 + I_3) \times (Min(C_1, C_2, C_3))}{E_1 + E_2 + E_3}$$
$$rank_5(n) = \frac{(R_1 \times R_2 \times R_3) \times (I_1 \times I_2 \times I_3) \times (Avg(C_1, C_2, C_3))}{E_1 \times E_2 \times E_3} \times 10^{-7}$$
$$rank_6(n) = \frac{(R_1 \times R_2 \times R_3) \times (I_1 \times I_2 \times I_3) \times (Min(C_1, C_2, C_3))}{E_1 \times E_2 \times E_3} \times 10^{-7}$$

In [300]:
#Defining the rank_1 function
def rank_1(num):
    score = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
                score += rice_scores.loc[i, "RICE"]
    return score

print(rank_1(1))

565.6666666666666


In [301]:
#defining the rank_2 function
def rank_2(num):
    numerator = 0
    denominator = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
                numerator += (rice_scores.loc[i, 'reach']*rice_scores.loc[i, 'impact']*rice_scores.loc[i, 'confidence'])
                denominator += rice_scores.loc[i, 'effort']
    return numerator/denominator

#check rank_2(1) "by hand"
rank_2_1 = ((170*1*0.8)+(220*3*0.5)+(80*3*0.5)+(300*1*0.8)+(200*2*0.5))/(4+2+2+1+3)
print(rank_2_1)

#check it against the function
print(rank_2(1))

85.5
85.5


In [302]:
#defining the rank_3 function
def rank_3(num):
    reach = 0
    impact = 0
    confidence = []
    effort = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
            reach = reach + rice_scores.loc[i, 'reach']
            impact = impact + rice_scores.loc[i, 'impact']
            confidence.append(rice_scores.loc[i, 'confidence'])
            effort = effort + rice_scores.loc[i, 'effort']
    return (reach*impact*statistics.mean(confidence))/effort

#check rank_3(1) "by hand"
rank_3_1 = ((170+220+80+300+200)*(1+3+3+1+2)*0.62)/(4+2+2+1+3)
print(rank_3_1)

#check it against the function
print(rank_3(1))

501.1666666666667
501.1666666666667


In [303]:
#defining the rank_4 function
def rank_4(num):
    reach = 0
    impact = 0
    confidence = []
    effort = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
            reach = reach + rice_scores.loc[i, 'reach']
            impact = impact + rice_scores.loc[i, 'impact']
            confidence.append(rice_scores.loc[i, 'confidence'])
            effort = effort + rice_scores.loc[i, 'effort']
    return (reach*impact*min(confidence))/effort

#check rank_4(1) "by hand"
rank_4_1 = ((170+220+80+300+200)*(1+3+3+1+2)*0.5)/(4+2+2+1+3)
print(rank_4_1)

#check it against the function
print(rank_4(1))

404.1666666666667
404.1666666666667


In [304]:
#defining the rank_5 function
def rank_5(num):
    reach = 1
    impact = 1
    confidence = []
    effort = 1
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
            reach = reach * rice_scores.loc[i, 'reach']
            impact = impact * rice_scores.loc[i, 'impact']
            confidence.append(rice_scores.loc[i, 'confidence'])
            effort = effort * rice_scores.loc[i, 'effort']
    return (reach*impact*statistics.mean(confidence))/(effort*10**7)

#check rank_5(1) "by hand"
rank_5_1 = ((170*220*80*300*200)*(1*3*3*1*2)*0.62)/((4*2*2*1*3*10**7))
print(rank_5_1)

#check it against the function
print(rank_5(1))

4173.84
4173.84


In [305]:
#defining the rank_6 function
def rank_6(num):
    reach = 1
    impact = 1
    confidence = []
    effort = 1
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
            reach = reach * rice_scores.loc[i, 'reach']
            impact = impact * rice_scores.loc[i, 'impact']
            confidence.append(rice_scores.loc[i, 'confidence'])
            effort = effort * rice_scores.loc[i, 'effort']
    return (reach*impact*min(confidence))/(effort*10**7)

#check rank_6(1) "by hand"
rank_6_1 = ((170*220*80*300*200)*(1*3*3*1*2)*0.5)/((4*2*2*1*3*10**7))
print(rank_6_1)

#check it against the function
print(rank_6(1))

3366.0
3366.0


In [306]:
#Defining a new DataFrame
#I had trouble adding the columns by doing tech_deps['rank_1'] = ... so here I'm creating a dictionary where columns for each rank function

#create a list with rank_i(n) for i = 1,..., 6 and n = 1,..., 5
rank_1_list = []
rank_2_list = []
rank_3_list = []
rank_4_list = []
rank_5_list = []
rank_6_list = []
for i in [1, 2, 3, 4, 5]:
    rank_1_list.append(rank_1(i))
    rank_2_list.append(rank_2(i))
    rank_3_list.append(rank_3(i))
    rank_4_list.append(rank_4(i))
    rank_5_list.append(rank_5(i))
    rank_6_list.append(rank_6(i))

tech_deps_dict = {'tech_dep':[1,2,3,4,5], 'rank_1':rank_1_list, 'rank_2':rank_2_list, 'rank_3':rank_3_list,
                 'rank_4':rank_4_list, 'rank_5':rank_5_list, 'rank_6':rank_6_list,}

tech_deps = pd.DataFrame.from_dict(tech_deps_dict)
print(tech_deps)

   tech_dep      rank_1      rank_2      rank_3      rank_4        rank_5  \
0         1  565.666667   85.500000  501.166667  404.166667  4.173840e+03   
1         2  535.616667   64.575000  563.853571  320.025000  9.704571e+06   
2         3  595.750000  108.833333  599.733333  346.000000  9.266400e+03   
3         4  256.200000   62.250000  202.666667  114.000000  1.013760e-01   
4         5  504.616667   62.093750  475.312500  396.093750  5.184000e+04   

         rank_6  
0  3.366000e+03  
1  5.508000e+06  
2  5.346000e+03  
3  5.702400e-02  
4  4.320000e+04  


In [307]:
#Gathering all my tables in one place for ease of reference
print(rice_scores)
print(tech_to_product)
print(tech_deps)

  product  reach  impact  confidence  effort        RICE  main_tech_dep  \
0      P1    170       1         0.8       4   34.000000              1   
1      P2    200       3         0.5       3  100.000000              2   
2      P3    220       3         0.5       2  165.000000              1   
3      P4     40       3         0.8       5   19.200000              4   
4      P5    500       3         0.3       2  225.000000              2   
5      P6     80       3         0.5       2   60.000000              1   
6      P7    300       1         0.8       1  240.000000              5   
7      P8    120       2         0.3       1   72.000000              3   
8      P9     25       3         0.5       2   18.750000              5   
9     P10    200       2         0.5       3   66.666667              5   

   tech_dep_2  tech_dep_3  
0         2.0         NaN  
1         5.0         3.0  
2         3.0         4.0  
3         5.0         2.0  
4         NaN         NaN  
5     

In [308]:
#I'm finding I need some summary statistics in the tech_to_product table to help make sense of the rankings. Let's try to add those.

def total_tech_reach(num):
    total = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
                total += rice_scores.loc[i, "reach"]
    return total    

def total_tech_impact(num):
    total = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
                total += rice_scores.loc[i, "impact"]
    return total

def avg_tech_confidence(num):
    list = []
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
            list.append(rice_scores.loc[i, 'confidence'])
    return statistics.mean(list)

def min_tech_confidence(num):
    list = []
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
            list.append(rice_scores.loc[i, 'confidence'])
    return min(list)

def total_tech_effort(num):
    total = 0
    for i in range(len(rice_scores)):
        if rice_scores.loc[i, 'main_tech_dep'] == num or rice_scores.loc[i, 'tech_dep_2'] == num or rice_scores.loc[i, 'tech_dep_3'] == num:
                total += rice_scores.loc[i, "effort"]
    return total

In [309]:
#Add columns to tech_to_product sheet
tech_to_product['total_reach'] = tech_to_product['tech'].apply(total_tech_reach)

tech_to_product['total_impact'] = tech_to_product['tech'].apply(total_tech_impact)

tech_to_product['avg_confidence'] = tech_to_product['tech'].apply(avg_tech_confidence)

tech_to_product['min_confidence'] = tech_to_product['tech'].apply(min_tech_confidence)

tech_to_product['total_effort'] = tech_to_product['tech'].apply(total_tech_effort)

print(tech_to_product)

   tech dependent_1 dependent_2 dependent_3 dependent_4 dependent_5  \
0     1          P1          P3          P6          P7         P10   
1     2          P1          P2          P4          P5          P8   
2     3          P2          P3          P7          P8          P9   
3     4          P3          P4          P8         NaN         NaN   
4     5          P2          P4          P6          P7          P9   

  dependent_6  total_reach  total_impact  avg_confidence  min_confidence  \
0         NaN          970            10        0.620000             0.5   
1          P9         1255            17        0.528571             0.3   
2         NaN          865            12        0.520000             0.3   
3         NaN          380             8        0.533333             0.3   
4         P10          845            15        0.600000             0.5   

   total_effort  
0            12  
1            20  
2             9  
3             8  
4            16  


In [310]:
#export the DataFrame to csv files
rice_scores.to_csv('rice_scores.csv', index = False)
tech_to_product.to_csv('tech_to_product.csv', index = False)
tech_deps.to_csv('tech_deps.csv', index=False)

In [314]:
# doing rank_2(4) 'by hand' to figure out how it shrinks less than rank_2(5)

rice_numerator_3 = rice_scores.loc[2, 'reach']*rice_scores.loc[2, 'impact']*rice_scores.loc[2,'confidence']
print(rice_numerator_3)
print(rice_scores.loc[2,'effort'])

rice_numerator_4  = rice_scores.loc[3, 'reach']*rice_scores.loc[3, 'impact']*rice_scores.loc[3,'confidence']
print(rice_numerator_4)
print(rice_scores.loc[3,'effort'])

rice_numerator_8 = rice_scores.loc[7, 'reach']*rice_scores.loc[7, 'impact']*rice_scores.loc[7,'confidence']
print(rice_numerator_8)
print(rice_scores.loc[7,'effort'])

330.0
2
96.0
5
72.0
1


In [313]:
# doing rank_2(5) 'by hand' to figure out why it shrinks so much

rice_numerator_2  = rice_scores.loc[1, 'reach']*rice_scores.loc[1, 'impact']*rice_scores.loc[1,'confidence']
print(rice_numerator_2)
print(rice_scores.loc[1,'effort'])

rice_numerator_4  = rice_scores.loc[3, 'reach']*rice_scores.loc[3, 'impact']*rice_scores.loc[3,'confidence']
print(rice_numerator_4)
print(rice_scores.loc[3,'effort'])

rice_numerator_6  = rice_scores.loc[5, 'reach']*rice_scores.loc[5, 'impact']*rice_scores.loc[5,'confidence']
print(rice_numerator_6)
print(rice_scores.loc[5,'effort'])

rice_numerator_7  = rice_scores.loc[6, 'reach']*rice_scores.loc[6, 'impact']*rice_scores.loc[6,'confidence']
print(rice_numerator_7)
print(rice_scores.loc[6,'effort'])

rice_numerator_9  = rice_scores.loc[8, 'reach']*rice_scores.loc[8, 'impact']*rice_scores.loc[8,'confidence']
print(rice_numerator_9)
print(rice_scores.loc[8,'effort'])

rice_numerator_10  = rice_scores.loc[9, 'reach']*rice_scores.loc[9, 'impact']*rice_scores.loc[9,'confidence']
print(rice_numerator_10)
print(rice_scores.loc[9,'effort'])

300.0
3
96.0
5
120.0
2
240.0
1
37.5
2
200.0
3
